In [3]:
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
from datetime import datetime
import scipy
from scipy.interpolate import interp1d
from scipy.interpolate import splrep, splev, CubicSpline, splprep
from scipy.signal import savgol_filter
from IPython.display import clear_output
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import shutil
%matplotlib inline
from scipy.interpolate import UnivariateSpline
import tkinter
from tkinter import filedialog
from scipy.stats import norm

In [4]:
class Processing():
    log = []
    mask_list = []
    mask_list_numbers = []
    sample_thickness = 38
    sample_area = 9.25
    n = 1
    m = 'all'
    
    folder_path = 'C://Users//Enric ST//Desktop//Results//MLCs of PST//MLC_2_2_5//PE_data_adiabatic/'
    
    folder_name = 'MLC_2_2_5_80V_330_275_probe_4Hz'
    
    data_path = f'{folder_path}{folder_name}/'
    
    # folders = ['Raw data', 'Column filter', 'Q1s', 'Fitted Q1s', 'P(E) matrix', 'P(T) matrix', 'P(S) matrix', 'dP/dT', 'delta S']
    folders = ['Zhopa1', 'Zhopa2', 'Zhopa3']
    
    directories = []
    maxes = []
    lengths = []
    
    lowest_max_E = None
    points = None
    x_sim = None
    
    filenames = glob.glob(f'{data_path}*.tab')
    
    def create_folder(self, name):
        new_folder_path = self.data_path + f'{name}/'
        self.log.append(f'Creating folder {name}')
        if not os.path.isdir(new_folder_path):
            os.mkdir(new_folder_path)
            
        self.directories.append(new_folder_path)
        self.log.append(f'New folder created. Path {new_folder_path}')
    def create_dirs(self):
        
        for folder in self.folders:
            self.create_folder(folder)
            
        self.filenames = glob.glob(f'{self.directories[0]}*.tab')
        
        self.filenames = sorted(self.filenames)
    
    def mask_list_gen(self):
        just_names = os.listdir(self.directories[0])
        
        for name in just_names:
            if name.endswith('.csv'):
                self.mask_list.append(float(name.replace('.csv','')))
                self.mask_list_numbers.append(float(name))
            if name.endswith('.tab'):
                self.mask_list.append(name.replace('.tab',''))
                self.mask_list_numbers.append(float(name))
        self.mask_list = sorted(self.mask_list)
        self.mask_list_numbers = sorted(self.mask_list_numbers)
    
    def column_filter(self, dataset, output_name):
        
        empties = []
        
        file = pd.read_table (dataset)
        if file.empty:
            self.log.append(f'Dataset {dataset} is empty. Cannot continue')
            empties.append(dataset)
        else:
        
            self.maxes.append(max(file['#Drive Voltage']))
            self.lengths.append(len(file))
                
            file.drop(['E_SI', 'P_SI','Point','Time (ms)'], axis = 1, inplace = True)
                
            file.insert(0, 'E_field (kV/cm)', (10**-5*file['#Drive Voltage']/\
                                                (self.sample_thickness*10**-6))) #field in kV/cm      
            file.drop(['#Drive Voltage'], axis = 1, inplace = True)
                
            file['Measured Polarization'] = 10**-6*file['Measured Polarization']/\
                (self.sample_area*10**-4)# normalising polarisation by sample ares in cm^2
                
            file.insert(1, 'Measured Polarization (C/m^2)', (file['Measured Polarization']))
            file.drop(columns = ['Measured Polarization'], inplace = True)
                
            file.to_csv(self.directories[1] + '/' + output_name + '.csv')
        
    def E_max_finder(self):
        self.log.append('Finding min E_max')
        self.points = min(self.points)
        self.lowest_max_E = min(self.maxes)
        self.log.append(f'Simulation points {min(self.points)}')
        self.log.append(f'Maximum electric field for simulation = {min(self.lowest_max_E)}')
    
    def Q1_select(self, file, output_name):
        
        self.log.append(f'Resolving Q1 for file {output_name}')
        
        file.iloc[0:(file['E_field (kV/cm)'].idxmax()),:]
        
        file.drop(Q1[Q1['E_field (kV/cm)'] < 0].index, inplace = True)
        file.drop(Q1[Q1['Measured Polarization (C/m^2)'] < 0].index, inplace = True)
        
        Q1 = file
        
        Q1_field = Q1['E_field (kV/cm)']
        Q1_field = Q1['E_field (kV/cm)'] - min(Q1['E_field (kV/cm)'])
        Q1_polarisation = Q1['Measured Polarization (C/m^2)']
    
        Q1 = Q1.iloc[::-1]    
        Q1.to_csv(f'{self.directories[2]}/{output_name}.csv')
        
        return Q1, Q1_field, Q1_polarisation
        
    def data_sim(self, x_data, y_data, smooth = 'No', fit_kind = 'cubic'):
        
        fitting_Q1 = pd.DataFrame()
        
        if fit_kind == 'cubic':
            
            fit = interp1d(x_data, y_data, kind = 'cubic', fill_value = 'extrapolate', bounds_error = False)
        else:
            fit = interp1d(x_data, y_data, kind = 'linear', fill_value = 'extrapolate', bounds_error = False)
        
        y_sim = fit(x_sim)
        
        fitting_Q1.insert(0, 'Fitted E_field (kV/cm)', x_sim)
        fitting_Q1.insert(len(fitting_Q1.columns),\
                                       'Fitted Polarisation (C/m^2)', y_sim)
      
        fitting_Q1.to_csv(f'{self.directories[3]}/{output_name}.csv')
        
        return y_sim

    def initial_processing(self):
        
        if self.lowest_max_E != None:
            self.E_max_finder()
        
        self.x_sim = np.linspace(0, 0.99*self.lowest_max_E, self.points)
        
        if self.m == 'all' or self.m >= len(self.filenames):
            m = len(self.filenames)
            
        self.log.append(f'Processing first {m} files')
            
        for i, dataset in enumerate(self.filenames[:m]):
        
            clear_output(wait = True)
            
            # file = self.column_filter(dataset, self.)
        
    
    

In [5]:
trial = Processing()


In [6]:
trial.create_dirs()

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//Enric ST//Desktop//Results//MLCs of PST//MLC_2_2_5//PE_data_adiabatic/MLC_2_2_5_80V_330_275_probe_4Hz/Zhopa1/'

In [38]:
print(trial.filenames)

[]


In [51]:
print(trial.directories)

['C://Users//Enric ST//Desktop//Results//MLCs of PST//MLC_2_2_5//PE_data_adiabatic/MLC_2_2_5_80V_330_275_probe_4Hz/Zhopa1/', 'C://Users//Enric ST//Desktop//Results//MLCs of PST//MLC_2_2_5//PE_data_adiabatic/MLC_2_2_5_80V_330_275_probe_4Hz/Zhopa2/', 'C://Users//Enric ST//Desktop//Results//MLCs of PST//MLC_2_2_5//PE_data_adiabatic/MLC_2_2_5_80V_330_275_probe_4Hz/Zhopa3/']
